# **Scikit Learn: Tips & Trick**

## *Pipeline, ColumnTransformer, CrossValidation, Iperparametri e Feature Selection*

## **Pipeline**

Finora abbiamo visto gli algoritmi come dei singoli blocchi a sè stanti.
Tuttavia, spesso questo non corrisponde a ciò che accade nella realtà: infatti, si parla di **processing pipeline**, intesa come *sequenza* di più algoritmi e metodi da applicare per trasformare dei dati e portarli all'elaborazione.

In tal senso, potremmo decidere di applicare "a cascata" diversi *transformer* e *stimatori* oppure, in maniera più compatta, sfruttare le potenzialità offerte dalla classe `Pipeline()` di *Scikit Learn*.

Per capire come questa funziona, facciamo un esempio. Immaginiamo di fare in modo che ad un algoritmo di clustering segua uno step di normalizzazione usando uno standard scaler. Abbiamo in tal senso due possibilità.

1. La prima è quella di usare il transformer e lo stimatore in cascata:

In [1]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X, y = make_blobs()
scaler = StandardScaler()
X_new = scaler.fit_transform(X)
kmeans = KMeans()
kmeans.fit_predict(X_new)

array([4, 2, 6, 6, 6, 2, 6, 2, 3, 5, 7, 0, 2, 5, 7, 1, 2, 5, 2, 1, 2, 2,
       1, 5, 0, 1, 5, 1, 1, 7, 6, 1, 7, 7, 7, 3, 2, 6, 2, 1, 0, 3, 5, 4,
       0, 2, 7, 5, 1, 7, 2, 2, 7, 1, 6, 1, 3, 0, 5, 2, 2, 3, 0, 0, 4, 7,
       1, 2, 1, 3, 1, 4, 5, 7, 0, 1, 7, 4, 5, 3, 1, 2, 5, 5, 7, 3, 7, 7,
       4, 0, 0, 1, 4, 5, 7, 0, 7, 6, 0, 4])

2. L'altra è quella di utilizzare un oggetto di tipo `Pipeline()`, che ci permette di indicare i singoli step da seguire nella nostra linea di elaborazione dati.

Notiamo come l'oggetto di classe Pipeline() implementi *nativamente* i metodi `fit`, `predict` e `transform` (più altri) per permettere l'uso di pipeline in cui lo step finale è un transformer o uno stimatore.

In [2]:
from sklearn.pipeline import Pipeline

pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('kmeans', KMeans())
])

pipe.fit_predict(X)

array([4, 5, 2, 2, 2, 5, 2, 5, 2, 7, 3, 1, 5, 7, 3, 6, 5, 7, 5, 6, 5, 5,
       0, 7, 1, 6, 7, 6, 6, 3, 2, 6, 3, 3, 3, 2, 2, 2, 5, 0, 1, 2, 7, 4,
       1, 5, 3, 7, 6, 3, 5, 5, 6, 6, 2, 0, 2, 1, 7, 5, 5, 2, 1, 1, 4, 3,
       0, 5, 0, 2, 0, 4, 7, 6, 1, 0, 3, 4, 7, 2, 6, 5, 7, 7, 3, 2, 3, 3,
       4, 1, 1, 6, 4, 7, 3, 1, 3, 2, 1, 4])

### **Accesso e modifica dei parametri degli stimatori**

Possiamo accedere ed *impostare manualmente* i parametri dei singoli stimatori e transformer dall'interno della pipeline.

Questo può essere fatto al momento dell'inizializzazione:

In [3]:
pipe = Pipeline(steps=[
    ('scaler', MinMaxScaler(feature_range=(0, 2))),
    ('kmeans', KMeans(n_clusters=3))
])

oppure successivamente mediante il comando `set_params` e la notazione `stimatore__parametro`:

In [4]:
pipe.set_params(kmeans__n_clusters=4)

Pipeline(steps=[('scaler', MinMaxScaler(feature_range=(0, 2))),
                ('kmeans', KMeans(n_clusters=4))])

Possiamo anche accedere al singolo stimatore come se la pipeline fosse un dizionario:

In [5]:
pipe['kmeans']

KMeans(n_clusters=4)

## **ColumnTransformer**

Abbiamo visto come i dataset che utilizziamo non abbiano sempre dati di *tipo uniforme* (ad esempio, tutti numerici o categorici), e che alle volte sia necessario applicare differenti trasformazioni a feature differenti.

Così come la pipeline di processing, anche questa operazione può essere svolta una feature alla volta o, in maniera estremamente più semplice, creando un oggetto di classe `ColumnTransformer()` il quale lavora sui *dataframe*.

Immaginiamo ad esempio di dover trasformare tutte le colonne del dataset tips, normalizzando quelle numeriche ed usando un `OrdinalEncoder()` per le categoriche.

Per farlo, possiamo usare un `ColumnTransformer()` in questo modo:

In [6]:
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

tips = sns.load_dataset('tips')
ct = ColumnTransformer(
    [('scaler', StandardScaler(), ['total_bill', 'tip']),
     ('encoder', OrdinalEncoder(), ['sex', 'smoker', 'day', 'time'])],
    remainder='passthrough'
)

ct.fit(tips)
ct.transform(tips)

array([[-3.14711305e-01, -1.43994695e+00,  0.00000000e+00, ...,
         2.00000000e+00,  0.00000000e+00,  2.00000000e+00],
       [-1.06323531e+00, -9.69205340e-01,  1.00000000e+00, ...,
         2.00000000e+00,  0.00000000e+00,  3.00000000e+00],
       [ 1.37779900e-01,  3.63355539e-01,  1.00000000e+00, ...,
         2.00000000e+00,  0.00000000e+00,  3.00000000e+00],
       ...,
       [ 3.24629502e-01, -7.22971264e-01,  1.00000000e+00, ...,
         1.00000000e+00,  0.00000000e+00,  2.00000000e+00],
       [-2.21286504e-01, -9.04025732e-01,  1.00000000e+00, ...,
         1.00000000e+00,  0.00000000e+00,  2.00000000e+00],
       [-1.13228903e-01,  1.24660453e-03,  0.00000000e+00, ...,
         3.00000000e+00,  0.00000000e+00,  2.00000000e+00]])

Ovviamente, anche un `ColumnTransformer()` può essere usato in una *pipeline*.

## **Cross-Validation (Validazione incrociata)**

Quando abbiamo parlato di preparazione dei dati abbiamo visto come sia necessario suddividere gli stessi in due insiemi, ovvero quello di *training* e quello di *validazione*, allo scopo di assicurarsi che il modello sia in grado di generalizzare.

Tuttavia, questa procedura spesso non è sufficiente, perché all'interno dei *dati di training* o *testing*, anche se scelti con meccanismi casuali, è possibile che siano presenti dei particolari meccanismi di generazione propri di quel sottinsieme. In altre parole, c'è il rischio che l'algoritmo vada comunque in *overfitting* sui dati di training!

Per ovviare a questa evenienza, molto spesso non ci si limita ad un'unica iterazione di addestramento, ma si effettua la procedura chiamata **$k$-fold cross validation**.

La **cross validation** consiste di **$K$** iterazioni, e prevede che l'intero set di dati sia suddiviso in $K$ porzioni, ognuna delle quali sarà usata ad ogni iterazione come test set, mentre la restante parte del dataset sarà utilizzata per il training. Dopo $K$ iterazioni, i risultati saranno quindi mediati tra loro, ed il modello risulterà essere più robusto all'overfitting. Questa procedura è descritta graficamente nella seguente immagine (presa direttamente dalla documentazione di Scikit Learn).

![cross_validation](Images\grid_search_cross_validation.png)

Ovviamente, in **Scikit Learn** è possibile effettuare la procedura di **cross validazione** utilizzando la funzione `cross_validate()`, che permette di effettuare $k$ round di cross-validazione di un algoritmo su un dataset.

In [7]:
from sklearn.model_selection import cross_validate

kmeans = KMeans()
cross_validate(kmeans, X, y, scoring=['adjusted_rand_score', 'adjusted_mutual_info_score'])

{'fit_time': array([0.06099629, 0.05899954, 0.05499959, 0.05200195, 0.0530026 ]),
 'score_time': array([0.00899839, 0.00700164, 0.0079968 , 0.0069983 , 0.006001  ]),
 'test_adjusted_rand_score': array([0.6312589 , 0.40451316, 0.57478006, 0.37709899, 0.53864832]),
 'test_adjusted_mutual_info_score': array([0.6561022 , 0.59673047, 0.67458282, 0.56921283, 0.64605025])}

Se ci si vuole concentrare su un'**unica metrica**, si può usare la funzione `cross_val_score()`:

In [8]:
from sklearn.model_selection import cross_val_score

cross_val_score(kmeans, X, y, scoring='adjusted_rand_score')

array([0.5351144 , 0.43507122, 0.57478006, 0.45052368, 0.53864832])

Infine, possiamo usare la funzione `cross_val_predict()` per ottenere i valori predetti a valle della cross-validazione:

In [9]:
from sklearn.model_selection import cross_val_predict

cross_val_predict(kmeans, X, y)

array([4, 1, 3, 3, 1, 1, 1, 1, 3, 2, 0, 2, 1, 6, 0, 7, 1, 4, 1, 7, 1, 1,
       6, 4, 4, 0, 7, 6, 0, 5, 3, 0, 0, 0, 5, 3, 1, 1, 3, 6, 3, 7, 3, 2,
       3, 0, 6, 3, 4, 6, 5, 0, 6, 4, 0, 4, 5, 3, 3, 0, 5, 2, 0, 0, 3, 6,
       1, 5, 1, 2, 1, 3, 0, 4, 0, 1, 4, 3, 3, 2, 7, 3, 5, 5, 1, 3, 1, 7,
       2, 5, 5, 7, 2, 5, 1, 5, 1, 6, 5, 2])

## **Ottimizzazione degli iperparametri**

Le funzioni che abbiamo visto finora hanno due *problemi*:

- il primo è che le funzioni di cross-validazione non restituiscono uno *stimatore fittato*;
- il secondo è che la valutazione di diversi **iperparametri** è delegata *manualmente* all'utente.

Per ovviare a questi problemi, esistono delle tecniche di *ottimizzazione degli iperparametri*, che permettono di esplorare una serie di valori per ogni parametro dell'algoritmo selezionato, valutando la combinazione degli stessi che produce i risultati migliori.

Esistono numerose tecniche di ottimizzazione degli iperparametri; in questo frangente, valuteremo la più semplice di tutte, ovvero la **grid search** (traducibile in italiano con "ricerca a griglia").

![gridsearch](Images\grid_search.png)

La tecnica della grid search è schematizzata nella precedente immagine.

Immaginiamo per semplicità che il nostro algoritmo abbia soltanto *due possibili parametri*, e che tutti i possibili valori che vogliamo esplorare possano essere disposti lungo una griglia.

All'interno di questa, i *valori di una metrica* a nostra scelta (ad esempio, l'accuratezza) si andranno a disporre secondo **picchi** e **valli**.

Il nostro obiettivo, in questo specifico caso, è ovviamente trovare il **massimo picco** della funzione che correla i valori dei parametri con l'accuratezza. Per farlo, la grid search prova ad eseguire l'algoritmo sui dati di training per ogni combinazione di parametri, fino a trovare la migliore possibile.

**Scikit Learn** implementa oggetti di classe `GridSearchCV()`, che prendono uno *stimatore* (o una pipeline), effettuano la crossvalidazione sui dati, e restituiscono quello con i parametri migliori dopo l'addestramento.

Ad esempio:

In [10]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'n_clusters': [3, 4, 5]
}
kmeans = KMeans()
est = GridSearchCV(kmeans, parameters)
est.fit(X)

GridSearchCV(estimator=KMeans(), param_grid={'n_clusters': [3, 4, 5]})

E' anche possibile usare la grid search in combinazione con con una pipeline:

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

pipe = Pipeline([
    ('pca', PCA()),
    ('kmeans', KMeans())
])

param_grid = {
    'pca__n_components': [2, 3],
    'kmeans__n_clusters': [3, 4, 5]
}

est = GridSearchCV(pipe, param_grid)
est.fit(X)

c:\Users\Marco Cecca\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Marco Cecca\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 684, in _fit_and_score
    estimator.fit(X_train, **fit_params)
  File "c:\Users\Marco Cecca\anaconda3\lib\site-packages\sklearn\pipeline.py", line 378, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\Users\Marco Cecca\anaconda3\lib\site-packages\sklearn\pipeline.py", line 336, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:\User

GridSearchCV(estimator=Pipeline(steps=[('pca', PCA()), ('kmeans', KMeans())]),
             param_grid={'kmeans__n_clusters': [3, 4, 5],
                         'pca__n_components': [2, 3]})

## **Feature Selection**

Concludiamo con un breve cenno alle tecniche di feature selection, che ci permettono di *isolare* le **feature** maggiormente significative all'interno del nostro dataset, *scartando* le altre. In tal senso, Scikit Learn ci offre un intero package, ovvero il `feature_selection`, operante in tal senso.

## **Esercizi**

In [12]:
import warnings
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder

def warn(*args, **kwargs):
    pass
warnings.warn = warn

### **Es 8.0**

Creiamo una pipeline di processing che, dati i dati relativi a conto e mance del dataset tips, e come label il giorno dello stesso, calcoli l'ARI a valle dell'applicazione di un'operazione di scaling prima dell'algoritmo di clustering. 

Successivamente, provare a modificare il numero di cluster, ricalcolando l'ARI.

In [13]:
# Caricamento del dataset

tips = sns.load_dataset('tips')
X = tips.loc[:, ('total_bill', 'tip')]
y = tips.loc[:, ('day')]

In [14]:

pipe = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('kmeans', KMeans())
])
y_pred = pipe.fit_predict(X)
print(adjusted_rand_score(y, y_pred))

pipe.set_params(kmeans__n_clusters=5)
y_pred = pipe.fit_predict(X)
print(adjusted_rand_score(y_pred, y))

9.800263629729834e-05
0.009089589270701748


### **Es 8.1**

Usiamo un column transformer per filtrare e pre-elaborare i dati contenuti nel dataset Titanic.

In particolare, selezioniamo i dati relativi all'età ed alla tariffa dei passeggeri, riempiendo eventuali dati mancanti e scalandoli nel range [0, 1], e codifichiamo i dati relativi alla sopravvivenza, classe, genere e porta di imbarcazione del passeggero mediante un `OrdinalEncoder()` seguito da un imputer.

Una volta completato il transformer, usiamolo in una pipeline di clustering.

In [15]:
df = pd.read_csv('Dataset/titanic.csv')

numerical_transformer = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline([
    ('scaler', OrdinalEncoder()),
    ('imputer', SimpleImputer())
])

ct = ColumnTransformer(
    [('num_transf', numerical_transformer, ['Age', 'Fare']),
    ('cat_transf', categorical_transformer, ['Survived', 'Pclass', 'Sex', 'Embarked'])],
    remainder='drop'
)

complex_pipe = Pipeline([
    ('transformer', ct),
    ('kmeans', KMeans())
])

labels = complex_pipe.fit_predict(df)

### **Es 8.2**

Utilizziamo la tecnica della *grid search* per automatizzare la prova fatta nell'esercizio Es 8.1. In particolare, andiamo a variare il numero di cluster (`n_clusters`) tra 3 ed 8 e l'algoritmo usato dal kmeans (algorithm) tra `lloyd` ed `elkan`.

In [16]:
param_grid_pipe = {
    'kmeans__n_clusters': list(range(3, 9)),
    'kmeans__algorithm': ['lloyd', 'elkan']
}
search = GridSearchCV(
    pipe,
    param_grid_pipe,
    scoring='adjusted_rand_score',
    n_jobs=-1,
    cv=10).fit(X.values)

### **Es 8.3**

Utilizziamo la tecnica di **feature selection** più semplice offerta da Scikit Learn, ovvero `VarianceThreshold()` per effettuare una procedura di feature selection sul dataset Titanic.

In tal senso, integriamo tale procedura nel transformer per i dati di tipo numerico usati nell'esercizio Es 8.1, e proviamo ad effettuare una grid search impostando due threshold (0 e 0.05) e facendo variare il numero di cluster tra 3 ed 8.

In [ ]:
fs = VarianceThreshold()

numerical_transformer = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', MinMaxScaler()),
    ('selector', VarianceThreshold())
])

categorical_transformer = Pipeline([
    ('scaler', OrdinalEncoder()),
    ('imputer', SimpleImputer())
])

ct = ColumnTransformer(
    [('num_transf', numerical_transformer, ['Age', 'Fare']),
    ('cat_transf', categorical_transformer, ['Survived', 'Pclass', 'Sex', 'Embarked'])],
    remainder='drop'
)

very_complex_pipe = Pipeline([
    ('transformer', ct),
    ('kmeans', KMeans())
])

param_grid_very_complex_pipe = {
    'transformer__num_transf__selector__threshold': [0.0, 0.05],
    'kmeans__n_clusters': list(range(3, 9)),
}

very_complex_search = GridSearchCV(
    very_complex_pipe,
    param_grid_very_complex_pipe,
    scoring='adjusted_rand_score',
    n_jobs=-1,
    cv=10).fit(df)